In [5]:
import pandas as pd
import requests

matches = pd.read_csv('WCMatches.csv')
matches.columns
matches[["winnerid","loserid"]]




,winnerid,loserid
0,7d05pfwn,jfvflnna
1,o8q1tnkp,o8q1tnkp
2,akq5t76h,akq5t76h
3,ng9mpb4s,ng9mpb4s
4,ng9mpb4s,ng9mpb4s
...,...,...
2783,dq44xncy,222vn7x7
2784,g7umra42,g7umra42
2785,g7umra42,222vn7x7
2786,g7umra42,dq44xncy


In [16]:
unique_ids = pd.concat([matches['winnerid'], matches['loserid']]).unique()
print(unique_ids.size)

1210


In [ ]:
import time

player_info = pd.DataFrame()
for player_id in unique_ids[1:2]:
    url_events = f"https://mtgeloproject.net/api/players/{player_id}/events"
    url_matches = f"https://mtgeloproject.net/api/players/{player_id}/matches"
    response_events = requests.get(url_events)
    response_matches = requests.get(url_matches)
    
    # Convert responses to JSON
    events_data = response_events.json()
    matches_data = response_matches.json()

    #flatten and rename events
    events_data_df = pd.DataFrame(events_data['data'])
    events_data_df = events_data_df.rename(columns={"code":"tournament_code"})

    # Flatten matches_data so each match is a row, including tournament code
    all_matches = []
    for tournament_code, matches_list in matches_data.items():
        for match in matches_list:
            match_copy = match.copy()
            match_copy['tournament_code'] = tournament_code
            all_matches.append(match_copy)
    matches_data_flat_df = pd.DataFrame(all_matches)

    # Expand own_elo and opp_data columns into separate columns and drop duplicate column
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('own_elo')).add_prefix('own_elo_'))
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('opp_data')).add_prefix('opp_'))
    matches_data_flat_df = matches_data_flat_df.drop(columns = "format")

    # Merge matches and events on event code
    merged_df = pd.merge(matches_data_flat_df, events_data_df, on="tournament_code", how="left", copy=False)
    #add player id
    merged_df.insert(loc=0, column= "player_id", value=player_id)
    
    ''' 
    TODO:
    Append merged_df to player_info or append/save to csv
    save player_info after each loop

    '''


    #time.sleep(0.5) #wait 0.5 second to not pummel the server


#player_info.to_csv("EloPlayerinfo.csv")
print(player_info)

Empty DataFrame
Columns: []
Index: []


In [ ]:
#flatten and rename events
events_data_df = pd.DataFrame(events_data['data'])
events_data_df = events_data_df.rename(columns={"code":"tournament_code"})

# Flatten matches_data so each match is a row, including tournament code
all_matches = []
for tournament_code, matches_list in matches_data.items():
    for match in matches_list:
        match_copy = match.copy()
        match_copy['tournament_code'] = tournament_code
        all_matches.append(match_copy)
matches_data_flat_df = pd.DataFrame(all_matches)

# Expand own_elo and opp_data columns into separate columns and drop duplicate column
matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('own_elo')).add_prefix('own_elo_'))
matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('opp_data')).add_prefix('opp_'))
matches_data_flat_df = matches_data_flat_df.drop(columns = "format")

# Merge matches and events on event code
merged_df = pd.merge(matches_data_flat_df, events_data_df, on="tournament_code", how="left", copy=False)

merged_df.insert(loc=0, column= "player_id", value=player_id)

# Save merged data
merged_df.to_csv("MergedEventsMatches.csv", index=False)